In [8]:
import numpy as np 
import xarray as xr
#from os.path import join 

#### Path variables

In [9]:
import os
os.getcwd()

'/home/kevin/Escritorio/Kevin/workspace/CRAI_hourly_infilling/WindData_Preprocessing/5.-Data_Split_train_val_test'

In [13]:
path = '/home/kevin/Escritorio/Kevin/workspace/CRAI_hourly_infilling/WindData_Preprocessing/'
input_path = "4.-Split_Data_Can_Pen/"
data_file = "cleaned_aemet_daily-wind_m_s_2011-2021_010x010_144_pen.nc"
mask_file = "aemet_mask_daily-wind_m_s_2011-2021_010x010_144_pen.nc"

exp_name_in_data = "windspeed" # wind speed
exp_name_out_data = "ws" # wind speed
exp_name_in_mask = "mask" # wind speed
exp_name_out_mask = "ws" # wind speed

#### Split data by years

In [14]:
def split_data_byyears(mask_path, data_path, years, opath):

    print(data_path)

    dataset = xr.open_dataset(data_path)
    mask_dataset = xr.open_dataset(mask_path)
    #dataset = dataset.rename({exp_name_in_data: exp_name_out_data}) #needed in the kadow code
    #mask_dataset = mask_dataset.rename({exp_name_in_mask: exp_name_out_mask}) #needed in the kadow code

    hdata = dataset['ws']
    hmask = mask_dataset['ws']

    print("Shape of the data: ", hdata.shape)
    print("Shape of the mask: ", hmask.shape)

    aemet_test_data = hdata.sel(time=slice(str(years[0]), str(years[1])))
    aemet_test_mask = hmask.sel(time=slice(str(years[0]), str(years[1])))
    
    print("")
    print("Shape of the aemet test data: ", aemet_test_data.shape)
    print("Shape of the aemet test mask: ", aemet_test_mask.shape)

    aemet_test_data.to_netcdf(opath + "aemet_daily-wind_m_s_" + str(years[0]) + "-" + str(years[1]) + "_010x010_144.nc")
    aemet_test_mask.to_netcdf(opath + "mask_aemet_daily-wind_m_s_" + str(years[0]) + "-" + str(years[1]) + "_010x010_144.nc")

    return hmask, hdata  

#### Main

In [15]:
if __name__ == '__main__':
    np.random.seed(42)

    #split_ratio = 7 / 9
    #split_test1 = 5 / 10
    #split_test2 = 2 / 3 

    years = [2011, 2021]


    output_path = "5.-Data_Split_train_val_test/Split_data_meters_second_" + str(years[0]) + "-" + str(years[-1]) + "/AEMET/"

    if not os.path.exists(path + output_path):
        os.makedirs(path + output_path)

    mask, data = split_data_byyears(path + input_path + mask_file, path + input_path + data_file, years, path + output_path)

/home/kevin/Escritorio/Kevin/workspace/CRAI_hourly_infilling/WindData_Preprocessing/4.-Split_Data_Can_Pen/cleaned_aemet_daily-wind_m_s_2011-2021_010x010_144_pen.nc
Shape of the data:  (4018, 144, 144)
Shape of the mask:  (4018, 144, 144)

Shape of the aemet test data:  (4018, 144, 144)
Shape of the aemet test mask:  (4018, 144, 144)


#### Check the ratio of Non-zero AEMET data

In [16]:
print("Number of AEMET data: ", data.count().item())
print("Number of Non-zero AEMET data: ", (data > 0.).sum().item())
print("Ratio of zero AEMET data: {}", 100 - (data > 0.).sum().item() * 100 / data.count().item(), "%")
print("Ratio of Non-zero AEMET data: {}", (data > 0.).sum().item() * 100 / data.count().item(), "%")

Number of AEMET data:  83317248
Number of Non-zero AEMET data:  1969022
Ratio of zero AEMET data: {} 97.63671742974516 %
Ratio of Non-zero AEMET data: {} 2.363282570254841 %


#### Check the ratio of Non-zero Mask data

In [17]:
print("Number of mask data: ", mask.count().item())
print("Number of Non-zero mask data: ", (mask > 0.).sum().item())
print("Ratio of zero mask data: ", 100 - (mask > 0.).sum().item() * 100 / data.count().item(), "%")
print("Ratio of Non-zero mask data: ", (mask > 0.).sum().item() * 100 / data.count().item(), "%")

Number of mask data:  83317248
Number of Non-zero mask data:  1976270
Ratio of zero mask data:  97.6280181505755 %
Ratio of Non-zero mask data:  2.3719818494245035 %
